TODO：

1.改成导航距离，选择heuristic=公交/步行/驾车距离

2.换个网好点的api

3.加cache

In [4]:
MAIMAI_URL="http://wc.wahlap.net/maidx/rest/location"

import requests
from bs4 import BeautifulSoup

import tqdm

import math
import geocoder as geocoder

import pandas as pd




#===pa===
url=MAIMAI_URL
 
headers = {
                'Cookie':'OCSSID=4df0bjva6j7ejussu8al3eqo03',
                'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
                '(KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
            }
res = requests.get(url,headers = headers)
soup = BeautifulSoup(res.content,"html.parser",from_encoding='gb18030')
all_maimai=eval(soup.get_text())




#====search====
#究极relax之我会飞 
#import tqdm

province=input("输入省/直辖市：")
search_result=[]
for maimai in all_maimai:
    if maimai['province']==province:
        search_result.append(maimai)
for i in range(len(search_result)):
    geo=geocoder.arcgis(search_result[i]['address']).latlng
    search_result[i]["coordinate"]=geo





address=input("输入地址：")
#target=geocoder.arcgis(u"浙江省杭州市秋溢路228号").latlng
target=geocoder.arcgis(address).latlng
targetx,targety=target
targetx=targetx*1000
targety=targety*1000

empty_list=[]

for i in range(len(search_result)):
    if type(search_result[i]['coordinate'])!=list:
        empty_list.append(i)
        continue
    x,y=search_result[i]['coordinate']
    x=1000*x
    y=1000*y
    dis=math.sqrt(pow(x-targetx,2)+pow(y-targety,2))
    
    search_result[i]['dis_to_target']=dis

for i in empty_list:
    tmp=search_result.pop(i)
    print("{}地图上找不到坐标，没算进去".format(tmp['arcadeName']))
    
print("到{}最近的maimaiDX排序如下：".format(address))
distance_ranking=sorted(search_result,key=lambda x:x['dis_to_target'])


#===show===

pd.DataFrame(distance_ranking)

输入省/直辖市：上海
输入地址：上海市人民广场地铁站
到上海市人民广场地铁站最近的maimaiDX排序如下：
   placeId  machineCount    id province        arcadeName              mall  \
0     1006             3  1006       上海           风云再起上海店           风云再起上海店   
1     1268             1  1268       上海         汤姆熊上海大悦城店         汤姆熊上海大悦城店   
2     1187             6  1187       上海     上海烈火游戏机娱乐有限公司     上海烈火游戏机娱乐有限公司   
3     1267             1  1267       上海        汤姆熊上海正大广场店        汤姆熊上海正大广场店   
4     1614             1  1614       上海      星际传奇瑞虹天地太阳宫店      星际传奇瑞虹天地太阳宫店   
5     1220             2  1220       上海      天天玩上海徐汇汇银广场店      天天玩上海徐汇汇银广场店   
6     1548             1  1548       上海         聚玩堂上海龙之梦店         聚玩堂上海龙之梦店   
7     1288             2  1288       上海       汤姆熊上海长宁龙之梦店       汤姆熊上海长宁龙之梦店   
8     1098             4  1098       上海  上海JOYPOLIS世嘉都市乐园  上海JOYPOLIS世嘉都市乐园   
9     1527             1  1527       上海         汤姆熊上海环球港店         汤姆熊上海环球港店   
10    1269             1  1269       上海     汤姆熊欢乐世界上海日月光店     汤姆熊欢乐世界上海日月光店 